<a href="https://colab.research.google.com/github/mohyunyang/MyStudy/blob/master/DL_NLP(Text_Preprocessing3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패딩(Padding)
- 자연어 처리를 하다보면 각 문장(또는 문서)은 서로 길이가 다를 수 있습니다. 
- 그런데 기계는 길이가 전부 동일한 문서들에 대해서는 하나의 행렬로 보고, 한꺼번에 묶어서 처리할 수 있습니다. 
- ★ 다시 말해 병렬 연산을 위해서 여러 문장의 길이를 임의로 동일하게 맞춰주는 작업이 필요할 때가 있습니다. 실습을 통해 이해해봅시다.

numpy로 패딩하기

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

# 단어 집합을 만들고, 정수 인코딩을 수행합니다.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_sentences) 
encoded = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded)

# 단어의 최대 길이 확인
max_len = max(len(item) for item in encoded)
print('최대 길이:',max_len)

In [ ]:
for sentence in encoded:
    while len(sentence) < max_len:
        sentence.append(0)

padded_np = np.array(encoded)
padded_np

## 케라스 전처리 도구로 패딩하기

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
encoded1 = tokenizer.texts_to_sequences(preprocessed_sentences)
print(encoded1)

In [ ]:
padded = pad_sequences(encoded1)
padded

In [ ]:
# Numpy로 패딩을 진행하였을 때와는 패딩 결과가 다른데 그 이유는 pad_sequences는 기본적으로 문서의 뒤에 0을 채우는 것이 아니라 앞에 0으로 채우기 때문입니다. 뒤에 0을 채우고 싶다면 인자로 padding='post'를 주면됩니다.

padded_b = pad_sequences(encoded1, padding='post')
padded_b

In [ ]:
# Numpy를 이용하여 패딩을 했을 때와 결과가 동일합니다. 실제로 결과가 동일한지 두 결과를 비교합니다.
(padded_b == padded_np).all()

길이가 5보다 짧은 문서들은 0으로 패딩되고, 기존에 5보다 길었다면 데이터가 손실됩니다. 가령, 뒤에서 두번째 문장은 본래 [ 7, 7, 3, 2, 10, 1, 11]였으나 현재는 [ 3, 2, 10, 1, 11]로 변경된 것을 볼 수 있습니다. 만약, 데이터가 손실될 경우에 앞의 단어가 아니라 뒤의 단어가 삭제되도록 하고싶다면 truncating이라는 인자를 사용합니다. truncating='post'를 사용할 경우 뒤의 단어가 삭제됩니다.

In [ ]:
padded_max = pad_sequences(encoded, padding='post', maxlen=5)
padded_max

In [ ]:
#  truncating='post'를 사용할 경우 뒤의 단어가 삭제
padded_max1 = pad_sequences(encoded, padding='post', truncating='post', maxlen=5)
padded_max1 truncating='post'를 사용할 경우 뒤의 단어가 삭제

# 한국어 전처리 패키지

## 1. PyKoSpacing

In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'

In [ ]:
new_sent = sent.replace(" ",'') # 띄어쓰기를 공백으로

In [ ]:
new_sent

In [ ]:
import tensorflow as tf
from pykospacing import Spacing

# tf.autograph.experimental.do_not_convert
spacing = Spacing() # 띄어쓰기 객체
kospacing_sent = spacing(new_sent)
print(sent)
print(kospacing_sent)

## 2. Py-Hanspell

In [ ]:
pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

In [ ]:
spelled_sent = spell_checker.check(new_sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)
print(kospacing_sent) # 앞서 사용한 kospacing 패키지에서 얻은 결과

In [ ]:
sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
sent_1 = "므리커럭이 나와쑈 우뤼가 다 치오끄요 바퀴뽈레 섀끠들이 케쇽 나호니끄 그 뜸을 막흐세욧"

## SOYNLP를 이용한 단어 토큰화

soynlp는 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저입니다. 비지도 학습으로 단어 토큰화를 한다는 특징을 갖고 있으며, 데이터에 자주 등장하는 단어들을 단어로 분석합니다. soynlp 단어 토크나이저는 내부적으로 단어 점수 표로 동작합니다. 이 점수는 응집 확률(cohesion probability)과 브랜칭 엔트로피(branching entropy)를 활용합니다.

In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Okt
tokenizer = Okt()
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정'))

In [ ]:
pip install soynlp

In [ ]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")


In [ ]:
# 훈련 데이터를 다수의 문서로 분리
corpus = DoublespaceLineCorpus("2016-10-20.txt")
len(corpus)

## SOYNLP의 브랜칭 엔트로피(branching entropy)

In [ ]:
# 훈련 데이터를 다수의 문서로 분리
corpus = DoublespaceLineCorpus("2016-10-20.txt")
len(corpus)

i = 0
for d in corpus:
  if len(d) > 0:
    print(i+1,d)
    i += 1
  if i == 10:
    break

print(type(corpus))

정상 출력되는 것을 확인하였습니다. soynlp는 학습 기반의 단어 토크나이저이므로 기존의 KoNLPy에서 제공하는 형태소 분석기들과는 달리 학습 과정을 거쳐야 합니다. 이는 전체 코퍼스로부터 응집 확률과 브랜칭 엔트로피 단어 점수표를 만드는 과정입니다. WordExtractor.extract()를 통해서 전체 코퍼스에 대해 단어 점수표를 계산합니다.

In [ ]:
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()
# 학습완료

## SOYNLP의 L tokenizer

한국어는 띄어쓰기 단위로 나눈 어절 토큰은 주로 L 토큰 + R 토큰의 형식을 가질 때가 많습니다. 예를 들어서 '공원에'는 '공원 + 에'로 나눌 수 있겠지요. 또는 '공부하는'은 '공부 + 하는'으로 나눌 수도 있을 것입니다. L 토크나이저는 L 토큰 + R 토큰으로 나누되, 분리 기준을 점수가 가장 높은 L 토큰을 찾아내는 원리를 가지고 있습니다.

In [ ]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

In [ ]:
word_score_table.items()

## 최대 점수 토크나이저
- 최대 점수 토크나이저는 띄어쓰기가 되지 않는 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저입니다. 띄어쓰기가 되어 있지 않은 문장을 넣어서 점수를 통해 토큰화 된 결과를 보겠습니다.

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

## SOYNLP를 이용한 반복되는 문자 정제
- SNS나 채팅 데이터와 같은 한국어 데이터의 경우에는 ㅋㅋ, ㅎㅎ 등의 이모티콘의 경우 불필요하게 연속되는 경우가 많은데 ㅋㅋ, ㅋㅋㅋ, ㅋㅋㅋㅋ와 같은 경우를 모두 서로 다른 단어로 처리하는 것은 불필요합니다. 이에 반복되는 것은 하나로 정규화시켜줍니다.

In [ ]:
from soynlp.normalizer import *
print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ', num_repeats=2))

print(repeat_normalize('졔하하하하하하하하하핫', num_repeats=2))
print(repeat_normalize('졔하하하하하하핫', num_repeats=2))
print(repeat_normalize('졔하하하하핫', num_repeats=2))

## Customized KoNLPy
- 영어권 언어는 띄어쓰기만해도 단어들이 잘 분리되지만, 한국어는 그렇지 않다고 앞에서 몇 차례 언급했었습니다. 한국어 데이터를 사용하여 모델을 구현하는 것만큼 이번에는 형태소 분석기를 사용해서 단어 토큰화를 해보겠습니다. 그런데 형태소 분석기를 사용할 때, 이런 상황에 봉착한다면 어떻게 해야할까요?

- 형태소 분석 입력 : '은경이는 사무실로 갔습니다.'
- 형태소 분석 결과 : ['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [ ]:
pip install customized_konlpy

In [ ]:
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.morphs('은경이는 사무실로 갔습니다.')

In [ ]:
twitter.add_dictionary('은경이', 'Noun') # 은 + 경이를 은경이로 토큰화

In [ ]:
twitter.morphs('은경이는 사무실로 갔습니다.')